In [ ]:
import pandas as pd
import numpy as np
import re, math
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving test.txt to test.txt
Saving train.txt to train.txt


In [ ]:
trainData = pd.read_csv("/content/train.txt", header = None)
testData = pd.read_csv("/content/test.txt", header = None)

In [ ]:
trainData.head()


,0
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger


In [ ]:
testData.head()


,0
0,im feeling rather rotten so im not very ambiti...
1,im updating my blog because i feel shitty;sadness
2,i never make her separate from me because i do...
3,i left with my bouquet of red and yellow tulip...
4,i was feeling a little vain when i did this on...


In [ ]:

def renameColumn(dataset):
  dataset.rename(columns = {0 : 'description'}, inplace = True)
  return dataset

In [ ]:

renameColumn(trainData)
renameColumn(testData)

,description
0,im feeling rather rotten so im not very ambiti...
1,im updating my blog because i feel shitty;sadness
2,i never make her separate from me because i do...
3,i left with my bouquet of red and yellow tulip...
4,i was feeling a little vain when i did this on...
...,...
1995,i just keep feeling like someone is being unki...
1996,im feeling a little cranky negative after this...
1997,i feel that i am useful to my people and that ...
1998,im feeling more comfortable with derby i feel ...


In [ ]:
def emotionSplitter(dataset):
    dataset[["text", "emotion"]] = dataset["description"].str.split(";", expand = True)
    dataset = dataset.drop("description", axis = 1)
    return dataset

In [ ]:

trainData = emotionSplitter(trainData)
testData = emotionSplitter(testData)
     

In [ ]:
trainData.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
encodedEmotions = {"anger": 0, "fear": 1, "joy": 2, "love": 3, "sadness": 4, "surprise": 5}


In [ ]:

def emotionEncoder(dataset):
    dataset.replace(encodedEmotions, inplace = True)
    return dataset

In [ ]:

emotionEncoder(trainData)
emotionEncoder(testData)

,text,emotion
0,im feeling rather rotten so im not very ambiti...,4
1,im updating my blog because i feel shitty,4
2,i never make her separate from me because i do...,4
3,i left with my bouquet of red and yellow tulip...,2
4,i was feeling a little vain when i did this one,4
...,...,...
1995,i just keep feeling like someone is being unki...,0
1996,im feeling a little cranky negative after this...,0
1997,i feel that i am useful to my people and that ...,2
1998,im feeling more comfortable with derby i feel ...,2


In [ ]:

def labelFeatureSplitter(dataset):
  features = dataset["text"].to_numpy()
  labels = dataset["emotion"].to_numpy()
  return (features, labels)

In [ ]:
(trainFeatures, trainLabels) = labelFeatureSplitter(trainData)
(testFeatures, testLabels) = labelFeatureSplitter(testData)
     

In [ ]:
stopWords = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 
              'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 
              'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'is', 's', 'am', 'or', 'who', 
              'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 
              'these', 'your', 'his', 'through', 'don', 'me', 'were', 'her', 'more', 'himself', 
              'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to',
              'ours', 'had', 'she', 'all', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 
              'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because',
              'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 
              'herself', 'has', 'just', 'where', 'only', 'myself', 'which', 'those', 'i', 
              'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 
              'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than',
              'ti','ame',' m','have','i','say','ames','someti'
              ]

termExpansionDict = {"won't":"will not", "can't":"can not","n't":" not", 
                          "'re":" are","'s been":" has been","'s":" is","'d":" would","'ll":" will",
                          "'t":" not", "ve":" have", "'m":" am", "im":"i am"}

In [ ]:
vowels = ['a','e','i','o','u']

def vowelFinder(word):
  for i in range(len(word)):
    if word[i] in vowels or (i > 0 and word[i]=='y' and word[i-1] not in vowels):
      return True
  return False


In [ ]:
def doubleConsonant(word):
  if len(word) < 2:
    return False
  if word[0] == word[1]:
    if word[0] not in vowels:
      return True
  return False

In [ ]:
def cvcFormater(word):
  if len(word) == 3:
    if (word[0] not in vowels) and word[1] in vowels and (word[2] not in vowels and word[2] not in ['w','x','y']):
      return True
  elif len(word)==4:
    if (word[1] not in vowels) or (word [1] == 'y' and word[0] in vowels):
          if word[2] in vowels and (word[3] not in vowels and word[3] not in ['w','x','y']):
            return True
  return False

In [ ]:
def mFinder(word):
  m = 0
  if len(word) < 2:
    return m
  for i in range(1,len(word)):
    v = word[i-1]
    c = word[i]
    if (v in vowels) and (c not in vowels) or (v not in vowels and v != 'y') and (c == 'y'):
      m += 1
  return m


In [ ]:
def wordStemmer(word):
  #steps

  #step 1
    #1a
  if word[-4:] == 'sses':
    word = word[:-4]+'ss'
  elif word[-3:] == 'ies':
    word = word[:-3]+'i'
  elif word[-2:] == 'ss':
    pass
  elif word[-1:] == 's':
    word = word[:-1]
    #1b
  is_b1 = False
  if word[-3:] == 'eed' and mFinder(word[:-3]) > 0:
    word = word[:-3]+'ee'
    is_b1 = True
  elif vowelFinder(word[:-2]) and word[-2:] == 'ed':
    word = word[:-2]
    is_b1 = True
  elif vowelFinder(word[:-3]) and word[-3:] == 'ing':
    word = word[:-3]
    is_b1 = True
      #cleanup (only if 1b is applied)
  if is_b1 == True:
    if word[-2:] == 'at':
      word = word[:-2]+'ate'
    elif word[-2:] == 'bl':
      word = word[:-2]+'ble'
    elif doubleConsonant(word[-2:]) and word[-1] not in ['l','s','z']:
      word = word[:-1]
    elif len(word) > 3 and cvcFormater(word[-4:]) and mFinder(word)==1:
      word = word + 'e'
    #1c
  if vowelFinder(word[:-1]) and word[-1] == 'y':
    word = word[:-1]+'i'
  
  #step 2
  if len(word) > 7 and word[-7:] == 'ational' and mFinder(word[:-7]) > 0: #Relational
    word = word[:-7]+'ate'
  elif len(word) > 7 and word[-7:] == 'ization' and mFinder(word[:-7]) > 0:
    word = word[:-7]+'ize'
  elif len(word) > 7 and word[-7:] == 'iveness' and mFinder(word[:-7]) > 0:
    word = word[:-7]+'ive'
  elif len(word) > 7 and word[-7:] == 'fulness' and mFinder(word[:-7]) > 0:
    word = word[:-7]+'ful'
  elif len(word) > 7 and word[-7:] == 'ousness' and mFinder(word[:-7]) > 0:
    word = word[:-7]+'ous'
  elif len(word) > 6 and word[-6:] == 'biliti' and mFinder(word[:-6]) > 0:
    word = word[:-6]+'ble'
  elif len(word) > 6 and word[-6:] == 'tional' and mFinder(word[:-6]) > 0:
    word = word[:-6]+'tion'
  elif len(word) > 5 and word[-5:] == 'entli' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'ent'
  elif len(word) > 5 and word[-5:] == 'ousli' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'ous'
  elif len(word) > 5 and word[-5:] == 'ation' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'ate'
  elif len(word) > 5 and word[-5:] == 'alism' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'al'
  elif len(word) > 5 and word[-5:] == 'aliti' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'el'
  elif len(word) > 5 and word[-5:] == 'iviti' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'ive'
  elif len(word) > 4 and word[-4:] == 'enci' and mFinder(word[:-4]) > 0:
    word = word[:-4]+'ence'
  elif len(word) > 4 and word[-4:] == 'anvi' and mFinder(word[:-4]) > 0:
    word = word[:-4]+'ance'
  elif len(word) > 4 and word[-4:] == 'izer' and mFinder(word[:-4]) > 0:
    word = word[:-4]+'ize'
  elif len(word) > 4 and word[-4:] == 'ator' and mFinder(word[:-4]) > 0:
    word = word[:-4]+'ate'
  elif len(word) > 4 and word[-4:] == 'abli' and mFinder(word[:-4]) > 0:
    word = word[:-4]+'able'
  elif len(word) > 4 and word[-4:] == 'alli' and mFinder(word[:-4]) > 0:
    word = word[:-4]+'al'
  elif len(word) > 3 and word[-3:] == 'eli' and mFinder(word[:-3]) > 0:
    word = word[:-4]+'e'

  #step 3
  if len(word) > 5 and word[-5:] == 'icate' and mFinder(word[:-5]) > 0 :
    word = word[:-5]+'ic'
  elif len(word) > 5 and word[-5:] == 'ative' and mFinder(word[:-5]) > 0:
    word = word[:-5]
  elif len(word) > 5 and word[-5:] == 'alize' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'al'
  elif len(word) > 5 and word[-5:] == 'iciti' and mFinder(word[:-5]) > 0:
    word = word[:-5]+'ic'
  elif len(word) > 4 and word[-4:] == 'ical' and mFinder(word[:-4]) > 0:
    word = word[:-4]+'ic'
  elif len(word) > 4 and word[-4:] == 'ness' and mFinder(word[:-4]) > 0:
    word = word[:-4]
  elif len(word) > 3 and word[-3:] == 'ful' and mFinder(word[:-3]) > 0:
      word = word[:-3]
  
  #step 4
  if len(word) > 5 and word[-5:] == 'ement'  and mFinder(word[:-5]) > 1:
    word = word[:-5]
  elif len(word) > 4 and word[-4:] == 'ance' and mFinder(word[:-4]) > 1:
    word = word[:-4]
  elif len(word) > 4 and word[-4:] == 'ence' and mFinder(word[:-4]) > 1:
    word = word[:-4]
  elif len(word) > 4 and word[-4:] == 'able' and mFinder(word[:-4]) > 1:
    word = word[:-4]
  elif len(word) > 4 and word[-4:] == 'ible' and mFinder(word[:-4]) > 1:
    word = word[:-4]
  elif len(word) > 4 and word[-4:] == 'ment' and mFinder(word[:-4]) > 1:
    word = word[:-4]
  elif len(word) > 4 and (word[-4:] == 'sion' or word[-4:] == 'tion') and mFinder(word[:-4]) > 1:
    word = word[:-4]
  elif len(word) > 3 and word[-3:] == 'ent' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 3 and word[-3:] == 'ive' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 3 and word[-3:] == 'ant' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 3 and word[-3:] == 'ism' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 3 and word[-3:] == 'ate' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 3 and word[-3:] == 'iti' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 3 and word[-3:] == 'ous' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 3 and word[-3:] == 'ize' and mFinder(word[:-3]) > 1:
    word = word[:-3]
  elif len(word) > 2 and word[-2:] == 'al' and mFinder(word[:-2]) > 1:
    word = word[:-2]
  elif len(word) > 2 and word[-2:] == 'er' and mFinder(word[:-2]) > 1:
    word = word[:-2]
  elif len(word) > 2 and word[-2:] == 'ic' and mFinder(word[:-2]) > 1:
    word = word[:-2]
  elif len(word) > 2 and word[-2:] == 'ou' and mFinder(word[:-2]) > 1:
    word = word[:-2]

  
  #step 5
    #5a
  if word[-1] == 'e' and mFinder(word[:-1]) > 1:
    word = word[:-1]
  elif len(word) > 4 and not (cvcFormater(word[-4:])) and word[-1:]=='e' and mFinder(word[:-1])==1:
    word = word[:-1]
   #5b
  if len(word) > 2 and doubleConsonant(word[-2:]) and word[-1] == 'l' and mFinder(word) > 1:
    word = word[:-1]

  return word

In [ ]:
def termExpander(text):
    for c in list(termExpansionDict.keys()):  
      text = text.replace(c,termExpansionDict[c])
    return text
def removeStopwords(inputText):
    words = termExpander(inputText).split()
    cleanWords = [word for word in words if word not in stopWords]
    cleanWords = " ".join(cleanWords)
    return cleanWords
def cleaner(sent):
    sent = re.sub("@[A-Za-z0-9]+","",sent)
    sent = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", sent)
    sent = " ".join(sent.split())
    sent = sent.replace("#", "").replace("_", " ")
    return sent
     

In [ ]:
trainData["text"] = trainData["text"].apply(removeStopwords)
testData["text"] = testData["text"].apply(removeStopwords)
     


In [ ]:

trainFeatures

array(['didnt feel humiliated',
       'go feeling hopeless damned hopeful around someone cares awake',
       'grabbing minute post feel greedy wrong', ...,
       'feel strong good o haverall', 'feel like rude comment glad',
       'know lot feel stupid portray'], dtype=object)

In [ ]:
print(set(trainLabels))


{0, 1, 2, 3, 4, 5}


In [ ]:
def NgramSplitter(text,ngram=1): 
  words=[word for word in text.split(" ")]
  ngramList = []
  for i in range(0, len(words) - ngram + 1):
    sent = ""
    for j in range(i, i+ngram):
      sent+=words[j];
      if(j < i+ ngram - 1):
        sent+=" "
    ngramList.append(sent)

  return ngramList

In [ ]:
ngramValue = int(input("ngram: "))


ngram: 1


In [ ]:
NgramSplitter('i am chandra shekar', 2)

['i am', 'am chandra', 'chandra shekar']

In [ ]:

def WordEmotionIndexer(sentList,ngram=1):
  count = 0
  wordEmotionIndex = {}
  for sent in sentList:
    count+=1
    for word in NgramSplitter(sent,ngram):
      if word not in wordEmotionIndex:
        wordEmotionIndex[word] = 1
      else:
        wordEmotionIndex[word] += 1
  return wordEmotionIndex, count

In [ ]:
index = {} 
count = {}
for i in range(6):
  index[i],count[i] = WordEmotionIndexer(trainData.loc[trainData["emotion"]  == i]["text"],ngramValue)
     

In [ ]:

sum(count.values())


16000

In [ ]:

for i in range(6):
  print(count[i])
     

2159
1937
5362
1304
4666
572


In [ ]:
priorProbs={}

for i in range(6):
  priorProbs[i] = count[i]/sum(count.values())

In [ ]:

print("Prior Probabilities: ")
priorProbs

Prior Probabilities: 


{0: 0.1349375, 1: 0.1210625, 2: 0.335125, 3: 0.0815, 4: 0.291625, 5: 0.03575}

In [ ]:
sum(priorProbs.values())

1.0

In [ ]:


testY = testData


In [ ]:
def EmotionPredictor(sent):
  i = 0
  emotionClass = -10000
  for emot in range(6):
    result = 0
    for word in NgramSplitter(sent,ngramValue):
      try:
        result += math.log((index[emot][word] +1)/len(index[emot]))
      except:
        result += math.log(1/ len(index[emot]))
    result += math.log(priorProbs[emot])
    if result > emotionClass:
      emotionClass = result
      i = emot
  return i
     

In [ ]:

EmotionPredictor("The boy is joyful")

2

In [ ]:

prediction = []
for sent in testY.text:
  prediction.append(EmotionPredictor(sent))

In [ ]:

counter = {}

for val in prediction:
    if val not in counter:
        counter[val] = 0
    counter[val] += 1
counter

{4: 815, 2: 798, 0: 152, 1: 144, 5: 37, 3: 54}

In [ ]:

confusion_matrix(testY["emotion"], prediction, labels=[0,1,2,3,4,5])

array([[143,   7,  27,   0,  90,   8],
       [  5, 123,  22,   1,  72,   1],
       [  0,   3, 640,   5,  40,   7],
       [  3,   0,  72,  48,  34,   2],
       [  1,   3,  17,   0, 558,   2],
       [  0,   8,  20,   0,  21,  17]])

In [ ]:
print(classification_report(testY["emotion"], prediction, labels=[0,1,2,3,4,5]))


              precision    recall  f1-score   support

           0       0.94      0.52      0.67       275
           1       0.85      0.55      0.67       224
           2       0.80      0.92      0.86       695
           3       0.89      0.30      0.45       159
           4       0.68      0.96      0.80       581
           5       0.46      0.26      0.33        66

    accuracy                           0.76      2000
   macro avg       0.77      0.58      0.63      2000
weighted avg       0.79      0.76      0.74      2000

